In [8]:
### IMPORTS
#
from flask import Flask, jsonify, render_template, request
from pandas.plotting import lag_plot
from sklearn.metrics import mean_squared_error
from sqlalchemy import create_engine
from sqlite3 import Error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
# import statsmodels.tsa.arima.model.ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
# from statsmodels.tsa.arima.model import ARIMAResults
from statsmodels.tsa.stattools import adfuller
import dataframe_image as dfi
import datetime
import glob
import joblib
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import sqlite3
import time
# END IMPORTS

In [9]:
database = r'crypto_db.sqlite'

def create_connection(db_file):
    """ create a database connection to the SQLite database specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn
# END create_connection


### RUN SQL COMMAND
#
def execute_sql_cmd(conn, command):
    """ run a sql command statement
    :param conn: Connection object
    :param execute_sql_cmd: run sql statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(command)
    except Error as e:
        print(e)
# END execute_sql_cmd


### LOAD SCHEMA
#
conn = create_connection(database)
# END LOAD SCHEMA

In [47]:
asset = 'brentoil'

path = f'models/{asset}/'

MODEL = joblib.load(f'{path}{asset}_model.smd')

conn = create_connection(database)
query = f"SELECT time_period_end, price_close from historic_data WHERE asset_id='{asset}'"

if conn is not None:
    result = pd.read_sql_query(query,conn)
else:
    print("Error! cannot create the database connection.")
conn.close()

forecast = MODEL.forecast(365)[0]
forecast

#MODEL.forecast() shows predicted value, std error of forecast, and confidence interval (lower & upper bounds)

newdates = result['time_period_end'].max()

C:\Users\jakeb\anaconda3\envs\data_analytics\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


In [53]:
import datetime

start_index = datetime(2020, 2, 5)
end_index = datetime(2021, 2, 5)
forecast = model_fit.predict(start=start_index, end=end_index)


# for i in args:
#     newdates = newdates.append({'time_period_end': i}, ignore_index=True)

TypeError: 'module' object is not callable

In [ ]:
#### PLOTLY EXAMPLE
##


import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure([

    go.Scatter(
        name='Actual',
        x=result.Date,
        y=result.price_close,
        mode='lines',
        marker=dict(color="#008080"),
        line=dict(width=1),
        showlegend=True
    ),
    go.Scatter(
        name='Prediction Forecast',
        x=result.Date+1,
        y=forecast[0],
        marker=dict(color="#FF8C00"),
        line=dict(width=1),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        showlegend=True
    )
])

fig.update_layout(
    yaxis_title='',
    title='',
    hovermode="x"
)

fig.show()